In [6]:
import os
import sys
sys.path.insert(0, os.path.dirname('C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part'))

import pandas as pd

from ml_part.random_forest.data_prep.preparation import DataPreparation
from ml_part.random_forest.train import RFTrain

CSV_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\remained_features_pKa_08.02_v4_fixed_distances_chirality.csv'
smiles_filepath = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\smiles_to_index.pkl'

dataPreparation = DataPreparation(CSV_PATH)

unimportant_features_to_drop = ['dipole_moment']
X, y = dataPreparation.prepare_data_for_RF(is_pKa=True,
                                           use_mandatory_features=True,
                                           is_remove_outliers=True,
                                           is_remove_nan=False,
                                           outliers_features_to_skip=unimportant_features_to_drop)

correlated_features = ['f_atom_fraction', 'naHRing', 'nFaRing', 'nFaHRing', 'tpsa+f']
ring_features_to_remain = ['nFRing', 'nHRing', 'nARing', 'nFHRing']
features_to_drop = []
for feature_name in X.columns:
    if feature_name in correlated_features:
        features_to_drop.append(feature_name)
    elif "ring" in feature_name.lower() and feature_name not in ring_features_to_remain:
        features_to_drop.append(feature_name)
    elif len(X[feature_name].unique()) == 1:
        print(f"feature without unique values: {feature_name}")
        features_to_drop.append(feature_name)
    # elif "angle" in feature_name or "distance" in feature_name:
    #     features_to_drop.append(feature_name)

X = X.drop(features_to_drop, axis=1)

rf_train = RFTrain(X=X, 
                   y=y,
                   smiles_filepath=smiles_filepath,
                   is_pKa=True,
                   k_folds=2)

y_train = rf_train.y_train
X_train = rf_train.X_train

y_test = rf_train.y_test
X_test = rf_train.X_test

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(len(train_df), len(test_df))

True
183
['RPCS', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5', 'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'f_atom_fraction', 'dihedral_angle', 'FPSA3', 'distance_between_atoms_in_cycle_and_f_group', 'angle_X1X2R2', 'nF', 'angle_R1X1R2', 'nFAHRing', 'nAHRing', 'chirality', 'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA', 'angle_R2X2R1', 'mol_num_cycles', 'naRing', 'nN', 'f_freedom', 'tpsa+f', 'nFRing', 'identificator', 'nO', 'distance_between_atoms_in_f_group_centers', 'angle_X2X1R1', 'nARing', 'nFARing', 'nC', 'nFHRing', 'f_to_fg', 'pKa', 'logP']
PBF outliers indexes: [40, 71, 127]
f_atom_fraction outliers indexes: [124]
FPSA3 outliers indexes: [40]
sasa outliers indexes: [127]
PNSA5 outliers indexes: [37, 38]
distance_between_atoms_in_f_group_centers outliers indexes: [35]
logP outliers indexes: [82, 83]
Remains rows:169, amount of features: 40
144 25


In [7]:
test_df

,RPCS,PBF,mol_weight,dipole_moment,PPSA5,avg_atoms_in_cycle,nHRing,cis/trans,dihedral_angle,FPSA3,...,nFRing,identificator,nO,distance_between_atoms_in_f_group_centers,angle_X2X1R1,nARing,nC,nFHRing,f_to_fg,pKa
26,16.590193,0.609605,138.114,0.428177,14.658366,0.0,0,0,NaN,0.044389,...,0,0,2,NaN,NaN,0,5,0,1.0,3.55
53,27.417416,0.753232,148.109,0.573753,16.455047,2.5,0,0,68.615541,0.045887,...,0,0,2,3.120047,117.436980,2,6,0,2.0,3.51
18,21.306553,0.590383,136.098,0.298947,13.476809,4.0,0,0,179.999999,0.046176,...,0,0,2,4.334857,145.566707,1,5,0,1.0,4.02
36,22.806203,0.698663,176.163,0.355824,14.159698,3.5,0,0,95.889724,0.040798,...,0,0,2,6.295501,149.193596,2,8,0,2.0,4.35
22,24.924560,0.615207,164.152,0.669361,14.044210,6.0,0,0,33.749623,0.043610,...,0,0,2,3.533381,99.144594,1,7,0,2.0,4.18
52,18.180349,0.720601,112.128,0.249203,15.401796,2.5,0,0,NaN,0.048901,...,0,0,2,NaN,NaN,2,6,0,0.0,4.61
154,0.000000,0.609558,135.158,0.436074,10.360490,6.0,0,0,89.969625,0.033363,...,0,1,0,4.403610,144.312238,1,6,0,2.0,9.12
168,1.869556,0.808327,147.169,0.639812,10.279217,3.5,0,0,74.878218,0.029960,...,0,1,0,5.360160,99.781096,2,7,0,2.0,9.58
162,84.836969,0.586977,57.096,0.146974,7.909767,3.0,0,0,NaN,0.040995,...,0,1,0,NaN,NaN,1,3,0,0.0,8.76
104,107.940433,0.752891,103.140,0.421122,10.588537,4.0,0,0,NaN,0.041355,...,0,1,0,NaN,NaN,1,5,0,1.0,8.10


In [9]:
train_df.to_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_pKa_v4_features_2.2.csv')
test_df.to_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_pKa_v4_features_2.2.csv')

In [5]:
train_df.columns

Index(['RPCS', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
       'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'dihedral_angle', 'FPSA3',
       'distance_between_atoms_in_cycle_and_f_group', 'angle_X1X2R2', 'nF',
       'angle_R1X1R2', 'chirality', 'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA',
       'angle_R2X2R1', 'mol_num_cycles', 'nN', 'f_freedom', 'nFRing',
       'identificator', 'nO', 'distance_between_atoms_in_f_group_centers',
       'angle_X2X1R1', 'nARing', 'nC', 'nFHRing', 'f_to_fg', 'fold_id', 'pKa'],
      dtype='object')

In [6]:
test_df.columns

Index(['RPCS', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
       'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'dihedral_angle', 'FPSA3',
       'distance_between_atoms_in_cycle_and_f_group', 'angle_X1X2R2', 'nF',
       'angle_R1X1R2', 'chirality', 'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA',
       'angle_R2X2R1', 'mol_num_cycles', 'nN', 'f_freedom', 'nFRing',
       'identificator', 'nO', 'distance_between_atoms_in_f_group_centers',
       'angle_X2X1R1', 'nARing', 'nC', 'nFHRing', 'f_to_fg', 'pKa'],
      dtype='object')

---

---

LogP

In [1]:
import os
import sys
sys.path.insert(0, os.path.dirname('C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part'))

import pandas as pd

from ml_part.random_forest.data_prep.preparation import DataPreparation
from ml_part.random_forest.train import RFTrain

CSV_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\remained_features_logP_08.02_v4_fixed_distances_chirality.csv'
smiles_filepath = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\smiles_to_index.pkl'

dataPreparation = DataPreparation(CSV_PATH)

unimportant_features_to_drop = ['dipole_moment']
X, y = dataPreparation.prepare_data_for_RF(is_pKa=False,
                                           use_mandatory_features=True,
                                           is_remove_outliers=True,
                                           is_remove_nan=False,
                                           outliers_features_to_skip=unimportant_features_to_drop)

correlated_features = ['f_atom_fraction', 'naHRing', 'nFaRing', 'nFaHRing', 'tpsa+f']
ring_features_to_remain = ['nFRing', 'nHRing', 'nARing', 'nFHRing']
features_to_drop = []
for feature_name in X.columns:
    if feature_name in correlated_features:
        features_to_drop.append(feature_name)
    elif "ring" in feature_name.lower() and feature_name not in ring_features_to_remain:
        features_to_drop.append(feature_name)
    elif len(X[feature_name].unique()) == 1:
        print(f"feature without unique values: {feature_name}")
        features_to_drop.append(feature_name)
    # elif "angle" in feature_name or "distance" in feature_name:
    #     features_to_drop.append(feature_name)

X = X.drop(features_to_drop, axis=1)

rf_train = RFTrain(X=X, 
                   y=y,
                   smiles_filepath=smiles_filepath,
                   is_pKa=False,
                   k_folds=2)

y_train = rf_train.y_train
X_train = rf_train.X_train

y_test = rf_train.y_test
X_test = rf_train.X_test

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(len(train_df), len(test_df))

True
183
['f_freedom', 'distance_between_atoms_in_cycle_and_f_group', 'PPSA5', 'mol_num_cycles', 'nFRing', 'nAHRing', 'angle_R1X1R2', 'nF', 'identificator', 'mol_weight', 'dipole_moment', 'nHRing', 'nO', 'PBF', 'nC', 'angle_X2X1R1', 'nARing', 'angle_R2X2R1', 'cis/trans', 'PNSA5', 'FPSA3', 'naRing', 'tpsa+f', 'mol_volume', 'RPCS', 'f_atom_fraction', 'GeomShapeIndex', 'WPSA5', 'TASA', 'f_to_fg', 'dihedral_angle', 'nFARing', 'distance_between_atoms_in_f_group_centers', 'avg_atoms_in_cycle', 'angle_X1X2R2', 'nFHRing', 'nFAHRing', 'chirality', 'pKa', 'logP']
mol_volume outliers indexes: [127]
f_atom_fraction outliers indexes: [124]
distance_between_atoms_in_f_group_centers outliers indexes: [ 35 167]
logP outliers indexes: [82, 83]
Remains rows:172, amount of features: 40
147 25


In [5]:
train_df.to_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_logP_v4_features_2.2.csv')
test_df.to_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_logP_v4_features_2.2.csv')